In [ ]:
import xarray as xr
import numpy as np
from google.cloud import storage
import fsspec
from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr
from pathlib import Path
import ujson
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scores
import pandas as pd
import seaborn as sns
import os
import eccodes
import dask
import cfgrib
import herbie.accessors
import regionmask
import xskillscore
dask.config.set({'temporary_directory': '/mnt/disks/data/'})
sns.set_theme(style='whitegrid',context='talk')

In [ ]:
cluster = LocalCluster(
    n_workers=8,
    threads_per_worker=2,
    memory_limit='5GiB',
)
client = Client(cluster)


# Define metadata that will be available in the yaml file

In [ ]:
event_begin = '2021-06-26 00:00'
event_end = '2021-06-30 00:00'
location_center = {'latitude': 47.6062, 'longitude': -122.3321}

In [ ]:
fs_read = fsspec.filesystem('gcs', anon=False, skip_instance_cache=True)
fs_local = fsspec.filesystem('')  
json_dir = 'assets/json/'
json_list = fs_local.glob(str(json_dir)+'PANG*_.json')
so = dict(mode='rb', anon=True, default_fill_cache=False, default_cache_type='first')

def convert_longitude_to_360(longitude):
    return longitude % 360

def generate_json_from_grap_nc(u,fs, fs_out):
    with fs.open(u, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, u, inline_threshold=300)

        file_split = u.split('/') # seperate file path to create a unique name for each json 
        model = file_split[1].split('_')[0]
        date_string = file_split[-1].split('_')[3]
        outf = f'{json_dir}{model}_{date_string}_.json'
        print(outf)
        with fs_out.open(outf, 'wb') as f:
            f.write(ujson.dumps(h5chunks.translate()).encode());


# Define metric functions

In [ ]:
#Intensity heat metrics

def threshold_weighted_rmse(da_fcst: xr.DataArray, da_obs: xr.DataArray, threshold: float, threshold_tolerance: float):
    mse = scores.continuous.tw_squared_error(da_fcst, 
                                             da_obs, 
                                             interval_where_one=(threshold, np.inf), 
                                             interval_where_positive=(threshold-threshold_tolerance, np.inf)
                                             )
    rmse = np.sqrt(mse)
    return rmse

def mae_max_of_max_temperatures(da_fcst: xr.DataArray, da_obs: xr.DataArray):
    mae = scores.continuous.mae(da_fcst.groupby('time.day').max().max(dim='time'),
                                da_obs.groupby('time.day').max().max(dim='time'))
    return mae

def mae_max_of_min_temperatures(da_fcst: xr.DataArray, da_obs: xr.DataArray):
    mae = scores.continuous.mae(da_fcst.groupby('time.day').min().max(dim='time'),
                                da_obs.groupby('time.day').min().max(dim='time'))
    return mae


#Duration heat metrics
def onset_above_85th_percentile(da_fcst: xr.DataArray, da_obs: xr.DataArray, da_clim_85th: xr.DataArray):
    def first_above_threshold(da, threshold):
        above_threshold = da > threshold
        first_time = above_threshold.argmax(dim='time')
        return first_time

    fcst_first_above = first_above_threshold(da_fcst, da_clim_85th)
    obs_first_above = first_above_threshold(da_obs, da_clim_85th)
    
    onset_me = (fcst_first_above - obs_first_above).astype(float)
    onset_me_hours = onset_me * np.timedelta64(1, 'h')
    return onset_me_hours

def mae_onset_and_end_above_85th_percentile(da_fcst: xr.DataArray, da_obs: xr.DataArray, da_clim_85th: xr.DataArray):
    def first_and_last_above_threshold(da, threshold):
        above_threshold = da > threshold
        first_time = above_threshold.argmax(dim='time')
        last_time = len(da['time']) - above_threshold[::-1].argmax(dim='time') - 1
        return first_time, last_time

    fcst_first_above, fcst_last_above = first_and_last_above_threshold(da_fcst, da_clim_85th)
    obs_first_above, obs_last_above = first_and_last_above_threshold(da_obs, da_clim_85th)
    
    onset_mae = np.abs(fcst_first_above - obs_first_above).astype(float)
    end_mae = np.abs(fcst_last_above - obs_last_above).astype(float)
    
    mean_absolute_error = ((onset_mae + end_mae) / 2) * np.timedelta64(1, 'h')
    return mean_absolute_error


